In [34]:
import pandas as pd
from unidecode import unidecode
import time

In [4]:
historic = pd.read_csv('pipeline_historic.csv', sep=';')

In [5]:
links = historic['link'].tolist()

In [6]:
links = [x for x in links if x[:4] == '2021']

In [ ]:
links

In [8]:
df_columns = pd.read_csv('raw_data/'+links[0], sep=';', nrows=1)
col_names = df_columns.columns.tolist()
col_names.append('EXTRA')

In [ ]:
small_dfs = []
soma = 0
for link in links:
    small_df = pd.read_csv('raw_data/' + link, sep=';', skiprows=2, names=col_names)
    print(len(small_df))
    soma+=len(small_df)
    small_dfs.append(small_df)
print(soma)

In [10]:
large_df = pd.concat(small_dfs, ignore_index=True)

In [11]:
len(large_df)

297584

In [12]:
large_df['SOLICITACAO'] = large_df[',SOLICITACAO'].str[-7:]

In [13]:
large_df = large_df.drop(',SOLICITACAO', 1)

C:\Users\fer-x\AppData\Local\Temp/ipykernel_36536/1612963592.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  large_df = large_df.drop(',SOLICITACAO', 1)


In [14]:
large_df['LOGRADOURO'] = [str(x).split(',')[0] for x in large_df['LOGRADOURO_ASS']]

In [15]:
large_df['NUMBER'] = [str(x).split(',')[1] if len(str(x).split(',')) > 1 else '' for x in large_df['LOGRADOURO_ASS']]

In [16]:
large_df['DATA'] = pd.to_datetime(large_df['DATA'])

In [17]:
large_df = large_df.loc[(large_df['DATA'] >= '2021-01-01') & (large_df['DATA'] < '2022-01-01')]

In [18]:
large_df['ASSUNTO'] = large_df['ASSUNTO'].apply(lambda x: unidecode(str(x)).strip().upper())

In [73]:
large_df.to_csv('compilado_2021.csv', sep=';', encoding='utf-8')

# Enviando dataframe para o googleSpreadsheet

In [20]:
# https://medium.com/@jb.ranchana/write-and-append-dataframes-to-google-sheets-in-python-f62479460cf0
# Imports necessarios para instalar os modulos usados no codigo caso ainda não tenha instalado
# import sys, subprocess
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gspread'])
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'gspread_dataframe'])
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pydrive'])

0

In [2]:
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
scopes = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

In [3]:
credentials = Credentials.from_service_account_file('curibita156credentials.json', scopes=scopes)

gc = gspread.authorize(credentials)

gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# open a google sheet
gs = gc.open_by_key('1UzTqciM1ACaBUYIt0P-MMHFHEbIux_ataERiDvdp3Ts')
# select a work sheet from its name
worksheet1 = gs.worksheet('Página1')

In [ ]:
## bloco de exemplo
# dataframe (create or import it)
#df = pd.DataFrame({'a': ['pple','airplane','alligato'], 'b': ['banana', 'ball', 'butterfly'], 'c': ['cantaloupe', 'crane', 'cat']})
# write to dataframe
# worksheet1.clear()
# set_with_dataframe(worksheet=worksheet1, dataframe=large_df, include_index=False,
# include_column_header=True, resize=True)

In [11]:
worksheet1.clear()

{'spreadsheetId': '1UzTqciM1ACaBUYIt0P-MMHFHEbIux_ataERiDvdp3Ts',
 'clearedRange': "'Página1'!A1:X268900"}

In [38]:
large_df_test = large_df.fillna('')
large_df_test['DATA'] = large_df_test['DATA'].astype(str)

In [ ]:
## Arquivo muito grande, nao rola enviar de uma vez
#worksheet1.update([large_df_test.columns.values.tolist()] + large_df_test.values.tolist())

In [39]:
large_df_test = large_df_test#.iloc[0:10000]

In [40]:
len(large_df_test)

268899

In [35]:
worksheet1.clear()

{'spreadsheetId': '1UzTqciM1ACaBUYIt0P-MMHFHEbIux_ataERiDvdp3Ts',
 'clearedRange': "'Página1'!A1:X268900"}

Como o arquivo é muito grande, fiz um looping para enviar as linhas aos poucos para o google spreadsheet
inseri um intervalo entre as requisições para não ultrapassar as cotas de requisição por minuto e por usuario
essas informações estao disponiveis n console.cloud.gogle.com/apis na aba de "APIs e serviços ativados"

In [ ]:
i = 0
j = 2000
#n = len(large_df_test)
n = len(large_df_test)
worksheet1.update([large_df_test.columns.values.tolist()])
while i < n:
    print(i)

    if i+j > n:
        j = n - i
    
    df_split = large_df_test.iloc[i:i+j]
    df_values = df_split.values.tolist()
    gs.values_append('Página1', {'valueInputOption': 'RAW'}, {'values': df_values})
    i = i+j    
    time.sleep(2.4)

